# Calculate the external dependency factor for issues <span style="color:red">after refinement</span> for the given report variables

In [1]:
# Change directory to workspace root so that relative path loads work correctly.
from IPython.display import Markdown as md
import datetime
import os
try:
    os.chdir(os.path.join(os.getcwd(), '../'))
    print(os.getcwd())
except:
    pass

from ia.common.jira.connection import connect
import ia.dependency.dashboard as dashboard
import ia.dependency.algo as dependency
import ia.dependency.metrics_store as metrics

report_start_time = datetime.datetime.now()
report_timestamp = f'`Report executed at {report_start_time:%Y-%m-%d %H:%M}`'
md(report_timestamp)

/Users/lswolkien/Devel/EngineeringMetrics


`Report executed at 2020-06-08 17:09`

# Report variables

In [2]:
conf_space_key = "~lswolkien"
conf_parent_page = "Reports"
conf_summary_page = "Independence summary"

projects = {
    'DANCOE': "DANCOE - Dependencies after refinement",
    'DANMR': "DANMR - Dependencies after refinement",
    'DANSDP': "DANSDP - Dependencies after refinement",
    'DANTF': "DANTF - Dependencies after refinement",
    'DANMIG': "DANMIG - Dependencies after refinement",
    'DANDI': "DANDI - Dependencies after refinement",
    "DANUNCO": "DANUNCO - Dependencies after refinement",
    'DANMRGCORE': "DANMRGCORE - Dependencies after refinement"
}

# Access variables

In [3]:
jira_url = os.environ['JIRA_URL']
jira_username = os.environ['JIRA_USERNAME']
jira_password = os.environ['JIRA_PASSWORD']

conf_url = os.environ['CONFLUENCE_URL']
conf_username = os.environ['CONFLUENCE_USERNAME']
conf_password = os.environ['CONFLUENCE_PASSWORD']

## Update metrics

In [4]:
jira =  connect(jira_url, basic_auth=(jira_username, jira_password))

# update metrics
for jira_project_id in projects.keys():
    jql = f'project = {jira_project_id} AND issuetype not in ("Test", "Sub-Task", "Release", "Risk", "Incident") and status not in ("Done", "In Analysis", "Closed", "Open")'
    p, all_issues, all_with_dep = dependency.dependency_factor(jira, jql)

    independency_factor = 100-p
    metrics.save('after refinement', jira_project_id, independency_factor, all_issues, all_with_dep)
    
    conf_page_title = projects[jira_project_id]
    metrics_history = metrics.read_independence_stats('after refinement', jira_project_id)

    dashboard.publish_dependency_report(
        conf_url,
        conf_username,
        conf_password,
        conf_space_key,
        conf_page_title,
        conf_parent_page,
        independency_factor,
        all_issues,
        all_with_dep,
        metrics_history
    ) 


## Update independence summary page

In [5]:
metrics_history = {}

for jira_project_id in projects.keys():
    metrics_history[jira_project_id] = metrics.read_independence_stats('after refinement', jira_project_id)

dashboard.publish_dependency_summary_report(
    conf_url,
    conf_username,
    conf_password,
    conf_space_key,
    conf_parent_page,
    conf_summary_page,
    projects,
    metrics_history,
    jira
)





In [6]:
report_end_time = datetime.datetime.now()
report_timestamp = f'` Report finished at {report_end_time:%Y-%m-%d %H:%M}, time elapsed: {report_end_time - report_start_time}`'
md(report_timestamp)


` Report finished at 2020-06-08 17:11, time elapsed: 0:02:25.606287`

## Tear down

In [7]:

import glob
files = glob.glob('*.png')
for filename in files:
    try:
        os.remove(filename)
    except:
        print("Error while deleting file : ", filename)